In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from config import v_user, v_password
import pandas as pd
import vertica_python
import matplotlib.pyplot as plt
conn_info = {'host': 'devlx207',
             'port': 5433,
             'user': v_user,
             'password': v_password,
             'database': 'advana',
             # 10 minutes timeout on queries
             'read_timeout': 600,
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': True}
             
# simple connection, with manual close
connection = vertica_python.connect(**conn_info)
cur = connection.cursor()

In [5]:
#Pull Down data from current model using SQL
query = '''
select
    a.name_polnum as polnum,
    a.name_firstname as fn,
    a.name_lastname as ln,
    a.name_idnumber as ssn,
    a.name_sex as sex,
    a.name_age as age,
    a.name_marital as marital,
    a.name_tobacco_rating as tob_rating,
    b.amount,
    b.cov_code,
    d.sum_entrydate,
    e.exp_premium,
    CASE
        WHEN e.premium_mode in ('MA','MS','M','SM','TM','5','TT','7','MQ') then 12*e.prod_premium
        WHEN e.premium_mode in('3','Q','SQ') then 4*e.prod_premium
        WHEN e.premium_mode in ('2','S') then 2*e.prod_premium
        ELSE e.prod_premium
    END as annual_prem,
    CASE
        WHEN d.sum_entrydate between cast('2009-09-14' as date) and cast('2009-10-16' as date)
        OR d.sum_entrydate between cast('2010-09-13' as date) and cast('2010-10-15' as date)
        OR d.sum_entrydate between cast('2011-09-12' as date) and cast('2011-10-14' as date)
        OR d.sum_entrydate between cast('2012-09-10' as date) and cast('2012-10-12' as date)
        OR d.sum_entrydate between cast('2013-09-09' as date) and cast('2013-10-11' as date)
        OR d.sum_entrydate between cast('2014-09-08' as date) and cast('2014-10-10' as date)
        OR d.sum_entrydate between cast('2015-09-14' as date) and cast('2015-10-16' as date)
        OR d.sum_entrydate between cast('2016-09-12' as date) and cast('2016-10-14' as date)
        THEN 1
        ELSE 0
    END as qb,
    CASE
        WHEN b.amount<50000 THEN 1
        ELSE 0
    END as small_pol,
    CASE
        WHEN a.name_age < 17 THEN 1
        ELSE 0
    END as minor,
    CASE
        WHEN d.sum_team in ('G','L') THEN 1
        ELSE 0
    END as brk,
    d.sum_status as status,
    d.sum_app_type as app_type,
    e.amount_1035,
    e.cash_with_app,
    e.dividend_mode,
    f.num_riders,
    f.total_rider_amt,
    f.WP,
    f.GIR,
    f.ALIR,
    f.RTR1,
    f.LISR,
    f.WMC,
    f.IPR,
    f.LTC,
    f.DBR,
    g.replacement_code,
    h.final_date,
    CASE
        WHEN a.name_age > 70 OR b.amount>5000000 THEN 2
        WHEN a.name_age BETWEEN 61 AND 70 THEN 1
        WHEN b.amount > 500000 AND a.name_age<=16 THEN 1
        WHEN b.amount > 2000000 AND a.name_age>=51 THEN 1
        WHEN b.amount > 3000000 AND a.name_age>=41 THEN 1 
        ELSE 0
    END as aps_strt,
    k.num_ratings,
    m.num_xclusions
from
    winrisk.Name a
LEFT JOIN    winrisk.summary d on a.name_polnum = d.sum_polnum
LEFT JOIN    winrisk.coverages b on a.name_polnum = b.policy_no
LEFT JOIN    winrisk.Billing e on a.name_polnum = e.policy_no
LEFT JOIN    winrisk.replacement_summary g on a.name_polnum = g.policy_no
LEFT JOIN    (select policy_no, count(*) as num_ratings 
                from winrisk.ratings group by policy_no ) k on a.name_polnum = k.policy_no
LEFT JOIN    (select policy_no, count(*) as num_xclusions 
                from winrisk.exclusion group by policy_no ) m on a.name_polnum = m.policy_no                
        
LEFT JOIN (
select count(*) as num_riders,
       sum(amount) as total_rider_amt,
       sum(case when rider_code = 'WP' then 1 else 0 end) as WP,
       sum(case when rider_code = 'GIR' then 1 else 0 end) as GIR,
       sum(case when rider_code = 'ALIR' then 1 else 0 end) as ALIR,
       sum(case when rider_code = 'RTR1' then 1 else 0 end) as RTR1,
       sum(case when rider_code = 'LISR' then 1 else 0 end) as LISR,
       sum(case when rider_code = 'WMC' then 1 else 0 end) as WMC,
       sum(case when rider_code = 'IPR' then 1 else 0 end) as IPR,
       sum(case when rider_code = 'LTC' then 1 else 0 end) as LTC,
       sum(case when rider_code = 'DBR' then 1 else 0 end) as DBR,
       policy_no 
from
    winrisk.riders
group by
        policy_no) f on a.name_polnum=f.policy_no
       
LEFT JOIN    (select
    policy_no,
    min(case when value2 in ('DECL','INCP','NTRT','RPTD','DROP','WDRN','NTKN') then date_occured else cast('9999-12-31' as date) end) as final_date
from
    winrisk.Policy_Events 
group by
    policy_no) h on a.name_polnum = h.policy_no
where
    a.name_role = 1
    and d.sum_app_type NOT IN ('SRV','TMP','INC')
    and sum_entrydate >= cast('2015-07-01' as date) ;'''
PolData = pd.read_sql(query, connection)
PolData.to_csv("PolicyDataTest.csv")

In [ ]:
and sum_entrydate < cast('2015-06-30' as date)

In [14]:
#Pull Down data from current model using SQL
query = '''
WITH rows AS(
    Select  a.name_polnum, a.name_firstname, a.name_lastname, a.name_idnumber, d.sum_entrydate, 
            row_number() OVER (PARTITION BY a.name_idnumber, a.name_lastname, a.name_firstname ORDER BY d.sum_entrydate) as rn
    From winrisk.Name a, winrisk.summary d 
    WHERE
    a.name_polnum=d.sum_polnum )

select
    a.name_polnum as polnum,
    a.name_firstname as fn,
    a.name_lastname as ln,
    a.name_idnumber as ssn,
    a.name_sex as sex,
    a.name_age as age,
    a.name_marital as marital,
    a.name_tobacco_rating as tob_rating,
    b.amount,
    b.cov_code,
    d.sum_entrydate,
    CASE
        WHEN d.sum_entrydate between cast('2009-09-14' as date) and cast('2009-10-16' as date)
        OR d.sum_entrydate between cast('2010-09-13' as date) and cast('2010-10-15' as date)
        OR d.sum_entrydate between cast('2011-09-12' as date) and cast('2011-10-14' as date)
        OR d.sum_entrydate between cast('2012-09-10' as date) and cast('2012-10-12' as date)
        OR d.sum_entrydate between cast('2013-09-09' as date) and cast('2013-10-11' as date)
        OR d.sum_entrydate between cast('2014-09-08' as date) and cast('2014-10-10' as date)
        OR d.sum_entrydate between cast('2015-09-14' as date) and cast('2015-10-16' as date)
        OR d.sum_entrydate between cast('2016-09-12' as date) and cast('2016-10-14' as date)
        THEN 1
        ELSE 0
    END as qb,
    CASE
        WHEN b.amount<50000 THEN 1
        ELSE 0
    END as small_pol,
    CASE
        WHEN a.name_age < 17 THEN 1
        ELSE 0
    END as minor,
    CASE
        WHEN d.sum_team in ('G','L') THEN 1
        ELSE 0
    END as brk,
    d.sum_status as status,
    d.sum_app_type as app_type,
    e.amount_1035,
    e.premium_mode,
    f.num_riders,
    f.total_rider_amt,
    g.replacement_code,
    h.iss_dt,
    h.rptd_dt,
    e.exp_premium,
    CASE
        WHEN e.premium_mode in ('MA','MS','M','SM','TM','5','TT','7','MQ') then 12*e.prod_premium
        WHEN e.premium_mode in('3','Q','SQ') then 4*e.prod_premium
        WHEN e.premium_mode in ('2','S') then 2*e.prod_premium
        ELSE e.prod_premium
    END as annual_prem,
    CASE
        WHEN a.name_age > 70 OR b.amount>5000000 THEN 2
        WHEN a.name_age BETWEEN 61 AND 70 THEN 1
        WHEN b.amount > 500000 AND a.name_age<=16 THEN 1
        WHEN b.amount > 2000000 AND a.name_age>=51 THEN 1
        WHEN b.amount > 3000000 AND a.name_age>=41 THEN 1 
        ELSE 0
    END as aps_strt,
    k.num_ratings,
    m.mult_apps,
    m.rn as num_apps,
    n.rpt_rate as agency_rpt_rate,
    p.agt_rpt_rate as agent_rpt_rate,
    p.agt_num_apps,
    p.length_of_service,
    q.ben_type,
    q.ben_relation,
    r.num_med_tests
from
    winrisk.Name a
    
LEFT JOIN    winrisk.summary d on a.name_polnum = d.sum_polnum
LEFT JOIN    winrisk.coverages b on a.name_polnum = b.policy_no
LEFT JOIN    winrisk.Billing e on a.name_polnum = e.policy_no
LEFT JOIN    winrisk.replacement_summary g on a.name_polnum = g.policy_no
LEFT JOIN    winrisk.beneficiary q on a.name_polnum=q.ben_polnum
LEFT JOIN    (select policy_no, count(*) as num_ratings 
                from winrisk.ratings group by policy_no ) k on a.name_polnum = k.policy_no
LEFT JOIN    (select
    policy_no,
    min(case when value2='MAIL' then date_occured else cast('9999-12-31' as date) end) as iss_dt,
    min(case when value2='RPTD' then date_occured else cast('9999-12-31' as date) end) as rptd_dt
from
    winrisk.Policy_Events 
group by
    policy_no) h on a.name_polnum = h.policy_no
    
LEFT JOIN (
select count(*) as num_riders,
       sum(amount) as total_rider_amt,
       policy_no 
from
    winrisk.riders
group by
        policy_no) f on a.name_polnum=f.policy_no

LEFT JOIN 
(       Select mc.rn, mc.name_polnum, mc.name_firstname, mc.name_lastname, mc.name_idnumber, 
        CASE 
            WHEN datediff("wk",mc.sum_entrydate,mp.sum_entrydate) < 1 THEN 1
            WHEN datediff("wk",mk.sum_entrydate,mc.sum_entrydate) < 1 THEN 1
            ELSE 0
        END as mult_apps
        From rows mc
        left join rows mp on mc.rn=mp.rn-1 and mc.name_idnumber=mp.name_idnumber and mc.name_firstname=mp.name_firstname and mc.name_lastname=mp.name_lastname
        left join rows mk on mc.rn=mk.rn+1 and mc.name_idnumber=mk.name_idnumber and mc.name_firstname=mk.name_firstname and mc.name_lastname=mk.name_lastname) m on
 a.name_firstname=m.name_firstname and a.name_lastname=m.name_lastname and a.name_idnumber=m.name_idnumber
 and m.name_polnum=a.name_polnum
 
 LEFT JOIN( select sum_agency_code,
       sum(case when sum_status='NTRT' then 1 else 0 end)/count(sum_status) as rpt_rate 
from
    winrisk.summary
where 
    sum_app_type NOT IN ('SRV','TMP','INC')
    and sum_entrydate >= cast('2012-01-01' as date) 
group by sum_agency_code ) n on d.sum_agency_code=n.sum_agency_code

 LEFT JOIN( select sum_agentcode, count(sum_status) as agt_num_apps,
       sum(case when sum_status='NTRT' then 1 else 0 end)/count(sum_status) as agt_rpt_rate,
       datediff('mm',min(sum_entrydate),max(sum_entrydate)) as length_of_service
from
    winrisk.summary
where 
    sum_app_type NOT IN ('SRV','TMP','INC')
    and sum_entrydate >= cast('2012-01-01' as date) 
group by sum_agentcode ) p on d.sum_agentcode=p.sum_agentcode

where
    a.name_role = 1
    and d.sum_app_type NOT IN ('SRV','TMP','INC')
    and datediff("yy",sum_entrydate,iss_dt) < 1000 
    and sum_entrydate >= cast('2014-06-01' as date) and sum_entrydate < cast('2015-12-31' as date);'''
PolData = pd.read_sql(query, connection)
PolData.to_csv("Stage2PolicyData.csv")